In [1]:
import argparse
import numpy as np
import torch

from agent.DiPo import DiPo
from agent.replay_memory import ReplayMemory, DiffusionMemory

from tensorboardX import SummaryWriter
import gym
import os


In [4]:
import gym
import torch
#env = gym.make("Ant-v4")

env=gym.make("Hopper-v3")


/opt/conda/envs/mujoco/lib/python3.9/site-packages/gym/envs/registration.py:555: UserWarning: WARN: The environment Hopper-v3 is out of date. You should consider upgrading to version `v4`.
  logger.warn(


DependencyNotInstalled: No module named 'mujoco_py'. (HINT: you need to install mujoco_py, and also perform the setup instructions here: https://github.com/openai/mujoco-py/.)

In [14]:
parser = argparse.ArgumentParser(description='Diffusion Policy')
parser.add_argument('--env_name', default="Hopper-v3",
                    help='Mujoco Gym environment (default: Hopper-v3)')
parser.add_argument('--seed', type=int, default=0, metavar='N',
                    help='random seed (default: 0)')

parser.add_argument('--num_steps', type=int, default=1000000, metavar='N',
                    help='env timesteps (default: 1000000)')

parser.add_argument('--batch_size', type=int, default=256, metavar='N',
                    help='batch size (default: 256)')
parser.add_argument('--gamma', type=float, default=0.99, metavar='G',
                    help='discount factor for reward (default: 0.99)')
parser.add_argument('--tau', type=float, default=0.005, metavar='G',
                    help='target smoothing coefficient(τ) (default: 0.005)')
parser.add_argument('--update_actor_target_every', type=int, default=1, metavar='N',
                    help='update actor target per iteration (default: 1)')

parser.add_argument("--policy_type", type=str, default="Diffusion", metavar='S',
                    help="Diffusion, VAE or MLP")
parser.add_argument("--beta_schedule", type=str, default="cosine", metavar='S',
                    help="linear, cosine or vp")
parser.add_argument('--n_timesteps', type=int, default=100, metavar='N',
                    help='diffusion timesteps (default: 100)')
parser.add_argument('--diffusion_lr', type=float, default=0.0003, metavar='G',
                    help='diffusion learning rate (default: 0.0003)')
parser.add_argument('--critic_lr', type=float, default=0.0003, metavar='G',
                    help='critic learning rate (default: 0.0003)')
parser.add_argument('--action_lr', type=float, default=0.03, metavar='G',
                    help='diffusion learning rate (default: 0.03)')
parser.add_argument('--noise_ratio', type=float, default=1.0, metavar='G',
                    help='noise ratio in sample process (default: 1.0)')

parser.add_argument('--action_gradient_steps', type=int, default=20, metavar='N',
                    help='action gradient steps (default: 20)')
parser.add_argument('--ratio', type=float, default=0.1, metavar='G',
                    help='the ratio of action grad norm to action_dim (default: 0.1)')
parser.add_argument('--ac_grad_norm', type=float, default=2.0, metavar='G',
                    help='actor and critic grad norm (default: 1.0)')

parser.add_argument('--cuda', default='cuda:0',
                    help='run on CUDA (default: cuda:0)')


_StoreAction(option_strings=['--cuda'], dest='cuda', nargs=None, const=None, default='cuda:0', type=None, choices=None, required=False, help='run on CUDA (default: cuda:0)', metavar=None)

In [4]:
args=parser.parse_known_args()[0]

In [5]:
args.env_name="Ant-v4"

In [6]:
device = torch.device(args.cuda)

dir = "record"
# dir = "test"
log_dir = os.path.join(dir, f'{args.env_name}', f'policy_type={args.policy_type}', f'ratio={args.ratio}', f'seed={args.seed}')
writer = SummaryWriter(log_dir)

# Initial environment
env = gym.make(args.env_name)


In [7]:
state_size = int(np.prod(env.observation_space.shape))
action_size = int(np.prod(env.action_space.shape))
print(action_size)

# Set random seed
torch.manual_seed(args.seed)
np.random.seed(args.seed)

8


In [8]:
memory_size = 1e6
num_steps = args.num_steps
start_steps = 10000
eval_interval = 10000
updates_per_step = 1
batch_size = args.batch_size
log_interval = 10

memory = ReplayMemory(state_size, action_size, memory_size, device)
diffusion_memory = DiffusionMemory(state_size, action_size, memory_size, device)


In [9]:
agent = DiPo(args, state_size, env.action_space, memory, diffusion_memory, device)

In [10]:
def evaluate(env, agent, writer, steps):
    print("eva")
    episodes = 10
    returns = np.zeros((episodes,), dtype=np.float32)

    for i in range(episodes):
        print("eva_i",i)
        state = env.reset()
        episode_reward = 0.
        done = False
        state=state[0]
        start=0
        while not done and start<=1000:
            start+=1
            #print("start",start)
            #print("step1")
            action = agent.sample_action(state, eval=True)
            #print("step2")
            next_state, reward, done, _,info = env.step(action)
            #print("done",done)
            episode_reward += reward
            #print("next_state",next_state)
            """
            if(len(next_state.shape)==1):
                state = np.expand_dims(next_state,axis=0)
            else:
            """
            state=next_state
        returns[i] = episode_reward

    mean_return = np.mean(returns)

    writer.add_scalar(
            'reward/test', mean_return, steps)
    print('-' * 60)
    print(f'Num steps: {steps:<5}  '
              f'reward: {mean_return:<5.1f}')
    print('-' * 60)


In [ ]:
steps = 0
episodes = 0
state_shape={}
while steps < num_steps:
    episode_reward = 0.
    episode_steps = 0
    done = False
    state = env.reset()
    episodes += 1
    state=state[0]
    start=0
    while not done and start<=1000:
        start+=1
        if start_steps > steps:
            action = env.action_space.sample()
        else:
            action = agent.sample_action(state, eval=False)
        next_state, reward, done, _ ,info= env.step(action)

        mask = 0.0 if done else args.gamma

        steps += 1
        episode_steps += 1
        episode_reward += reward

        agent.append_memory(state[0], action, reward, next_state, mask)

        if steps >= start_steps:
            #print("train")
            agent.train(updates_per_step, batch_size=batch_size, log_writer=writer)

        if steps % eval_interval == 0:
            evaluate(env, agent, writer, steps)
            # self.save_models()
            done =True

        state = next_state

    # if episodes % log_interval == 0:
    #     writer.add_scalar('reward/train', episode_reward, steps)
    print("step",steps)
    print(f'episode: {episodes:<4}  '
        f'episode steps: {episode_steps:<4}  '
        f'reward: {episode_reward:<5.1f}')

step 37
episode: 1     episode steps: 37    reward: -2.3 
step 52
episode: 2     episode steps: 15    reward: -4.2 
step 249
episode: 3     episode steps: 197   reward: 8.5  
step 310
episode: 4     episode steps: 61    reward: 35.3 
step 344
episode: 5     episode steps: 34    reward: -7.6 
step 367
episode: 6     episode steps: 23    reward: -11.5
step 458
episode: 7     episode steps: 91    reward: -62.4
step 514
episode: 8     episode steps: 56    reward: -44.2
step 548
episode: 9     episode steps: 34    reward: -4.4 
step 581
episode: 10    episode steps: 33    reward: -11.4
step 634
episode: 11    episode steps: 53    reward: -9.2 
step 673
episode: 12    episode steps: 39    reward: -16.4
step 706
episode: 13    episode steps: 33    reward: -6.9 
step 742
episode: 14    episode steps: 36    reward: -6.2 
step 827
episode: 15    episode steps: 85    reward: -129.2
step 886
episode: 16    episode steps: 59    reward: -10.9
step 1047
episode: 17    episode steps: 161   reward: -18

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f211f7f9cd0>>
Traceback (most recent call last):
  File "/opt/conda/envs/mujoco/lib/python3.9/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
KeyboardInterrupt: 

KeyboardInterrupt



In [ ]:
state

In [ ]:
state.shape

In [ ]:
action

In [18]:
reward

0.41524802225522506

In [19]:
self.memory.idx

NameError: name 'self' is not defined